In [3]:
from mcp.server import Server
import mcp.types as types

import os
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# Database configuration
DB_CONFIG_COLUMNAR = {
    'host': os.getenv("DB_HOST"),
    'database': 'ambient_sensors_columnar',
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'port': 5432
}

conn = psycopg2.connect(**DB_CONFIG_COLUMNAR)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

cur = conn.cursor()

In [4]:
app = Server("sensor-data-server")

In [26]:
def create_sensor_dict(results, description):
    sensor_dict = {}
    for row in results:
        sensor_id = row[0]
        sensor_info = {description[i]: row[i] for i in range(1, len(description))}
        sensor_dict[sensor_id] = sensor_info
    return sensor_dict

def list_sensors() -> dict:
    cur.execute("SELECT * FROM sensors")
    results = cur.fetchall()
    description = [d.name for d in cur.description]
    resp_dict = create_sensor_dict(results, description)
    return resp_dict

In [27]:
@app.list_tools()
async def list_tools() -> list[types.Tool]:
    return [
        types.Tool(
            name="list_sensors",
            description="Get list of available sensors from the database",
            inputSchema={
                "type": "object",
                "properties": {},
                "required": []
            }
        )
    ]

@app.call_tool()
async def call_tool(name: str, arguments: dict) -> list[types.TextContent]:
    if name == "list_sensors":
        sensors = list_sensors()
        result_text = str(sensors)
        
        return [types.TextContent(
            type="text",
            text=result_text
        )]
    
    raise ValueError(f"Unknown tool: {name}")

[(1, 'esp32_c6_sensor_hub', 'bmp280', 'home', {'type': 'multi_value', 'fields': {'humidity': {'unit': '%RH', 'accuracy': 3, 'max_range': 100, 'min_range': 0, 'metric_type': 'humidity'}, 'pressure': {'unit': 'Pa', 'accuracy': 12, 'max_range': 110000, 'min_range': 30000, 'metric_type': 'pressure'}, 'temperature': {'unit': '°C', 'accuracy': 1, 'max_range': 85, 'min_range': -40, 'metric_type': 'temperature'}}, 'location': 'home', 'description': 'BME280 environmental sensor (pressure, temperature, humidity)'}, datetime.datetime(2025, 10, 23, 13, 6, 48, 700103, tzinfo=datetime.timezone.utc)), (2, 'esp32_c6_sensor_hub', 'scd30', 'home', {'type': 'multi_value', 'fields': {'co2': {'unit': 'ppm', 'accuracy': 30, 'max_range': 10000, 'min_range': 400, 'metric_type': 'co2'}, 'humidity': {'unit': '%RH', 'accuracy': 3, 'max_range': 100, 'min_range': 0, 'metric_type': 'humidity'}, 'temperature': {'unit': '°C', 'accuracy': 0.4000000059604645, 'max_range': 70, 'min_range': -40, 'metric_type': 'temperatu

In [12]:
cur.close()
conn.close()

In [20]:
description

(Column(name='sensor_id', type_code=23),
 Column(name='device_id', type_code=1043),
 Column(name='sensor_type', type_code=1043),
 Column(name='location', type_code=1043),
 Column(name='metadata', type_code=3802),
 Column(name='installed_date', type_code=1184))

In [22]:
results[0]

(1,
 'esp32_c6_sensor_hub',
 'bmp280',
 'home',
 {'type': 'multi_value',
  'fields': {'humidity': {'unit': '%RH',
    'accuracy': 3,
    'max_range': 100,
    'min_range': 0,
    'metric_type': 'humidity'},
   'pressure': {'unit': 'Pa',
    'accuracy': 12,
    'max_range': 110000,
    'min_range': 30000,
    'metric_type': 'pressure'},
   'temperature': {'unit': '°C',
    'accuracy': 1,
    'max_range': 85,
    'min_range': -40,
    'metric_type': 'temperature'}},
  'location': 'home',
  'description': 'BME280 environmental sensor (pressure, temperature, humidity)'},
 datetime.datetime(2025, 10, 23, 13, 6, 48, 700103, tzinfo=datetime.timezone.utc))